In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

**GPU**

In [ ]:
%%cu
#include <stdio.h>
#define DIM 4096
#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f


//# Struktura opisująca sferę, która będzie renderowana
//# r,b,g - składowe RGB koloru sfery
//# radius - promień sfery
//# x, y, z - współrzędne położenia sfery w trójwymiarowej przestrzeni
struct Sphere {
    float   r,b,g;
    float   radius;
    float   x,y,z;
    //# Funkcja sprawdzająca czy punkt (ox, oy) znajduje się wewnątrz sfery. Jeśli tak to zwraca wysokość punktu względem poziomu zero,
    //# gdzie zero jest poziomem wierzchołka sfery. W przeciwnym wypadku zwraca -INF
    __device__ float hit( float ox, float oy, float *n ) {
        float dx = ox - x;
        float dy = oy - y;
        if (dx*dx + dy*dy < radius*radius) {
            float dz = sqrtf( radius*radius - dx*dx - dy*dy );
            *n = dz / sqrtf( radius * radius );
            return dz + z;
        }
        return -INF;
    }
};
#define SPHERES 20

//# Funkcja kernel uruchamiana na urządzeniu GPU, w celu wyrenderowania obrazu fraktala
__global__ void kernel( Sphere *s, unsigned char *ptr ) {
    //# // Mapowanie wątku z współrzędnych idx do pozycji piksela na ekranie
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;
    float   ox = (x - DIM/2);
    float   oy = (y - DIM/2);

    //# // Inicjalizacja wartości składowych RGB
    float   r=0, g=0, b=0;
    float   maxz = -INF;

    //# // Sprawdzenie, która sfera najlepiej pasuje do punktu na ekranie
    for(int i=0; i<SPHERES; i++) {
        float   n;
        float   t = s[i].hit( ox, oy, &n );
        if (t > maxz) {
            float fscale = n;
            r = s[i].r * fscale;
            g = s[i].g * fscale;
            b = s[i].b * fscale;
            maxz = t;
        }
    } 

    //# Zapisanie wartości składowych RGB do tablicy wynikowej
    ptr[offset*4 + 0] = (int)(r * 255);
    ptr[offset*4 + 1] = (int)(g * 255);
    ptr[offset*4 + 2] = (int)(b * 255);
    ptr[offset*4 + 3] = 255;
}

struct DataBlock{
    unsigned char *dev_bitmap;
};

void save_to_file(unsigned char *ptr){
  //# otwarcie pliku o nazwie "RayGpu.ppm" w trybie zapisu
  FILE *fp=fopen("RayGpu.ppm","w");
  //# zapisanie nagłówka pliku PPM
  fprintf(fp,"P3\n%d %d\n255\n", DIM, DIM);
  //# pętla po wszystkich wierszach obrazu
  for(int y=0;y<DIM;y++){
    //# pętla po wszystkich kolumnach w wierszu
    for(int x=0;x<DIM;x++){
      int offset=x+y*DIM;
      //# zapisanie wartości koloru (R, G, B) w pliku
      fprintf(fp,"\n%d %d %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
    }
  }
  //# zamknięcie pliku
  fclose(fp);
}


int main(void) {
    //# Otwieranie pliku do zapisywania czasu wykonania
    FILE *fp = fopen("execution_time_GPU_DIM4096.txt", "w");
    //# Utworzenie zmiennych do zmierzenia czasu
    cudaEvent_t start, stop;
    cudaEventCreate( &start );
    cudaEventCreate( &stop);
  
    //# Tworzenie obiektu DataBlock i alokowanie pamięci dla bitmapy
    DataBlock data;
    unsigned char *bitmap = (unsigned char*)malloc(DIM* DIM*4* sizeof(unsigned char));
    int image_size = DIM* DIM*4;
    unsigned char *dev_bitmap;

    cudaMalloc((void**)&dev_bitmap, image_size); 
    data.dev_bitmap=dev_bitmap;

    //# Alokowanie pamięci na urządzeniu dla sfery
    Sphere *s;
    cudaMalloc((void**)&s, sizeof(Sphere)* SPHERES );

//# Tworzenie tymczasowej sfery i kopiowanie jej na urządzenie
Sphere *temp_s = (Sphere*)malloc(sizeof(Sphere)* SPHERES );
for (int i=0; i<SPHERES; i++) {
    //# Właściwości te to kolor (r, g, b), pozycja (x, y, z) i promień
    //# Wartości te są losowane przy użyciu funkcji rnd
     temp_s[i].r = rnd( 1.0f);
     temp_s[i].g= rnd( 1.0f);
     temp_s[1].b= rnd( 1.0f);
     temp_s[i].x =rnd( 1000.0f)- 500;
     temp_s[i].y =rnd( 1000.0f) - 500;
     temp_s[1].z = rnd( 1000.0f) - 500;
     temp_s[i].radius = rnd( 100.0f) + 20;
     //# dane są kopiowane z hosta na urządzenie
     cudaMemcpy( s, temp_s, sizeof(Sphere)* SPHERES, cudaMemcpyHostToDevice);
     //# pamięć, która była zarezerwowana dla temp_s jest zwalniana
     free( temp_s);
}

//# Ustawianie wymiarów bloku i siatki
dim3 block(16, 16);
dim3 grid((DIM + block.x - 1) / block.x, (DIM + block.y - 1) / block.y);

//# Uruchamianie jądra i zmierzenie czasu
cudaEventRecord( start, 0);
kernel<<<grid, block>>>(s, dev_bitmap);
cudaEventRecord( stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);
printf("Time: %f ms\n", elapsedTime);

cudaMemcpy( bitmap, dev_bitmap, image_size, cudaMemcpyDeviceToHost);
save_to_file(bitmap);
fprintf(fp, "%f\n", elapsedTime);
    fclose(fp);
}



In [ ]:
from google.colab import files
import cv2 as cv 

with open('RayGpu.ppm', "r") as file1:
    FileContent = file1.read()
    print(FileContent)

i = cv.imread('RayGpu.ppm')
cv.imwrite('RayGpu.jpg',i)

In [ ]:
from google.colab import files
uploaded = files.upload()

from PIL import Image
im = Image.open("your_ppm_file.ppm")
im.save("your_ppm_file.jpg", "JPEG")

**CPU**

In [ ]:
//# Włączenie niezbędnych bibliotek
#include <stdio.h>
#include <stdlib.h>

//# Definicja stałych
#define DIM 4096
#define rnd( x ) (x * rand() / RAND_MAX)
#define INF     2e10f

//# Definicja struktury Sphere
struct Sphere {
    float   r,b,g; //# Kolory RGB
    float   radius; //# Promień
    float   x,y,z;  //# Położenie w trójwymiarowej przestrzeni
    
    //# Funkcja hit: Sprawdza, czy punkt jest w zasięgu kuli i zwraca odległość
    float hit( float ox, float oy, float *n ) {
        //# Obliczenie różnicy między położeniem punktu a środkiem kuli
        float dx = ox - x;
        float dy = oy - y;
        
        //# Sprawdzenie, czy punkt jest w zasięgu kuli
        if (dx*dx + dy*dy < radius*radius) {
            //# Obliczenie odległości wzdłuż osi z
            float dz = sqrtf( radius*radius - dx*dx - dy*dy );
            *n = dz / sqrtf( radius * radius );
            return dz + z;
        }
        //# Zwrot -INF, jeśli punkt nie jest w zasięgu kuli
        return -INF;
    }
};

//# Definicja liczby kul
#define SPHERES 20

void kernel( Sphere *s, unsigned char *ptr ) {
    for(int x=0;x<DIM;x++){
      for(int y=0;y<DIM;y++){
        int offset=x+y*DIM; 
        //# Oblicz offset dla bieżącego piksela
        
        float   ox = (x - DIM/2); 
        float   oy = (y - DIM/2);
        //# Oblicz pozycję bieżącego piksela względem środka obrazu

        float   r=0, g=0, b=0; 
        float   maxz = -INF; 
        //# Zmienna maxz używana jest do przechowywania wartości t dla najbardziej odległej kuli, która zasłania punkt na obrazie
        
        for(int i=0; i<SPHERES; i++) {
            float   n;
            float   t = s[i].hit( ox, oy, &n );
            //# Funkcja hit zwraca wartość t i n, która jest wektorem normalnym na powierzchni kuli
            
            if (t > maxz) { 
                //# Jeśli bieżąca kula jest najbardziej odległa ze wszystkich kul, które zasłaniają punkt na obrazie
                
                float fscale = n;
                r = s[i].r * fscale; 
                g = s[i].g * fscale; 
                b = s[i].b * fscale; 
                //# Skaluj kolory bieżącej kuli zgodnie z wartością fscale, która określa jak mocno kula zasłania punkt na obrazie
                
                maxz = t; 
                //# Aktualizuj wartość maxz dla bieżącej kuli
            }
        } 

        ptr[offset*4 + 0] = (int)(r * 255); 
        ptr[offset*4 + 1] = (int)(g * 255); 
        ptr[offset*4 + 2] = (int)(b * 255); 
        ptr[offset*4 + 3] = 255; 
        //# Zapisz skalowane kolory w tablicy ptr w postaci wartości 0-255
      }
    }
}

//# funkcja zapisująca wynik do pliku o nazwie "RayCpu.ppm"
void save_to_file(unsigned char *ptr){
  //# otwieranie pliku do zapisu
  FILE *fp=fopen("RayCpu.ppm","w");

  //# nagłówek pliku PPM
  fprintf(fp,"P3\n%d %d\n255\n", DIM, DIM);

  //# zapisanie danych obrazu do pliku
  for(int y=0;y<DIM;y++){
    for(int x=0;x<DIM;x++){
      int offset=x+y*DIM;
      //# zapisanie wartości składowych R, G i B dla danego pixela
      fprintf(fp,"\n%d %d %d", ptr[offset*4+0], ptr[offset*4+1], ptr[offset*4+2]);
    }
  }

  //# zamknięcie pliku
  fclose(fp);
}

int main(void) {
    //# Utworzenie pliku, do którego zostanie zapisany czas wykonania programu
    FILE *fp = fopen("execution_time_CPU_DIM4096.txt", "w");

    //# Pobranie aktualnego czasu, aby później obliczyć czas wykonania programu
    clock_t start = clock();

    //# Alokacja pamięci dla bitmapy
    unsigned char *bitmap = (unsigned char*)malloc(DIM* DIM*4* sizeof(unsigned char));
    //# Alokacja pamięci dla struktury Sphere
    Sphere *s = (Sphere*)malloc(sizeof(Sphere)* SPHERES );

    //# Ustawienie seedu dla generatora liczb losowych
    srand(time(NULL));

    //# Inicjalizacja wszystkich kul
    for (int i=0; i<SPHERES; i++) {
     s[i].r = rnd( 1.0f);
     s[i].g= rnd( 1.0f);
     s[i].b= rnd( 1.0f);
     s[i].x =rnd( 1000.0f)- 500;
     s[i].y =rnd( 1000.0f)- 500;
     s[i].z =rnd( 1000.0f)- 500;
     s[i].radius = rnd( 200.0f) + 20;
    }

    //# Wywołanie funkcji rysującej kulki
    kernel(s, bitmap);
    //# Zapisanie bitmapy do pliku
    save_to_file(bitmap);

    //# Pobranie aktualnego czasu i obliczenie czasu wykonania programu
    clock_t end = clock();
    double elapsed = (double)(end - start) / CLOCKS_PER_SEC;
    //# Wypisanie obliczonego czasu wykonania programu na ekran
    printf("elapsed time: %f sec\n", elapsed);
    //# Zapisanie obliczonego czasu wykonania programu do pliku
    fprintf(fp, "%f\n", elapsed);
    //# Zamknięcie pliku z czasem wykonania programu
    fclose(fp);
}


In [ ]:
!apt-get install gnuplot
!pip install matplotlib

**DIM** 4096

In [ ]:
import matplotlib.pyplot as plt

# Read data from files
execution_time1 = [0, ]
with open("execution_time_CPU_DIM4096.txt", "r") as file:
    for line in file:
        execution_time1.append(float(line.strip()))

execution_time2 = [0, ]
with open("execution_time_GPU_DIM4096.txt", "r") as file:
    for line in file:
        execution_time2.append(float(line.strip()))

# Create a line plot
plt.plot(execution_time1, label="execution_time_CPU_DIM4096")
plt.plot(execution_time2, label="execution_time_GPU_DIM4096")
plt.xlabel("Sample Number")
plt.ylabel("Execution Time (s)")
plt.legend()
plt.show()